
Neda Jabbari;
Sep 2019



### Multiclass Logistic Regression and Genomics

##### Scenario #1: 
A customer walks into a car dealership to buy a car. Given the customer's age and budget what are the chances that they will buy that specific BMW that the dealer showed them?  

##### Scenario #2: 
A customer walks into a car dealership to buy a car. Given the customer's age and budget what is the likelihood of buying a particular model that the dealer showed them (with choices of Toyota, Chevrolet and BMW)? 

If you have ever wondered how you could use statistics to answer such questions, "logistic regression" is one answer. At times, we need to know the likelihood of a certain outcome of a dependent categorical variable (e.g. buying the particular BMW) given a set of features (customer's budget and age). As a binary regression, logistic regression estimates the relationship between independent variables and a certain output of a binary variable. Other times, we are interested in the likelihood of any of the multiple outcomes (e.g. buying any of the certain car models). In this case, a logistic regression is extended to several classes or events to figure out the likelihood of each. We refer to this as multiclass logistic regression. 

##### Objective:

Based on my interest in genomics and certain years of experience working on Borrelia genomes, I decided to see if the size and GC content of bacterial chromosome assembly can help tease apart bacterial subgroups. Let me give a little background on the question: Chromosomes are made of DNA and DNA consists of 4 bases of G, C, T and A. A chromosome GC content is the percentage of G and C bases to the total bases in the chromosome. 


##### Data Source:
I extracted the size and GC content of bacterial chromosome level assemblies from NCBI genome database. (https://www.ncbi.nlm.nih.gov/genome). I limited my data to bacteria with human host and one scaffold assembly. In addition, I only considered the three subgroups of Gammaproteobacteria, Firmicutes and Actinobacteria since the number of submissions in these subgroups were relatively comparable. 

In [ ]:
#import relevant packages
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import roc_curve, auc
from sklearn import metrics
from scipy import interp
from itertools import cycle
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv('prokaryotes.csv')
df['Organism Groups'].value_counts()

In [ ]:
df.head()

In [ ]:
#Figure out what Organism Groups have above 100 available submissions in dataset . 
summary = df.groupby('Organism Groups')[['Size(Mb)','GC%']].describe()
bigenough = summary[summary['Size(Mb)']['count'] > 100]
bigenough

In [ ]:
# Keep 'Gammaproteobacteria','Actinobacteria','Firmicutes' with >100 chromosome level assemblies.
df['Organism Groups'] = [i.split(sep=';')[-1] for i in df['Organism Groups']]
bacclasses = ['Gammaproteobacteria','Actinobacteria','Firmicutes']
inbacclass = df['Organism Groups'].isin(bacclasses)
df = df[inbacclass]

In [ ]:
# Keep those with one scaffold in the assembly.
df = df[df['Scaffolds']==1]
df.groupby('Organism Groups').count()

In [ ]:
df.duplicated().sum()

In [ ]:
df['Organism Groups'].value_counts()

In [ ]:
#Replace the subgroups Firmicutes, Actinobacteria and Gammaproteobacteria with 0, 1, and 2 accordingly. 
bacecode = dict(zip(df['Organism Groups'].unique(),range(3)))
df['Organism Groups'] = df['Organism Groups'].replace(bacecode)

In [ ]:
df['Organism Groups'].value_counts()

It is time to get started for modeling. First, I prepare an X set as a dataframe with features "Size(Mb)" and "GC%" and a y series for Organism Groups and then split the data into train and test using a test size of 0.3.

In [ ]:
y = df['Organism Groups']
features = ['Size(Mb)','GC%']
X = df[features]

In [ ]:
# split X and y into train and test 
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=0)

#### Multiclass logistic regression
Here, I fit the model to train set and predict on test set:

In [ ]:
logreg = LogisticRegression() #instantiate the model with the default parameters
result = logreg.fit(X_train,y_train)   
y_pred=logreg.predict(X_test) 

##### But how well did the algorithm perform?

Let’s look at the coefficient of the features in my function. The coefficients represent log odds and are separately calculated for each independent variable in each class of the dependent variable.

These log odds follow from the the “logistic function” thresholding used in the model https://en.wikipedia.org/wiki/Logistic_function.

##### What do the classifier coefficients mean and how can I interpret them to evaluate the performance of my model?

Each estimated coefficient is the expected change in the log odds of being in the relevant class for a unit increase in the predictor variable while the other variables are held constant. For example, the first coefficient of 0.032 (top left value in the above array) means that if we hold GC content constant, we will see 0.032% increase in the odds of getting classified as subgroup Firmicutes for a one-unit increase in the assembly size (exp(0.032) = 1.032). In other words, the odds for subgroup Firmicutes is 0.032% higher than the odds for other groups.
The intercept is the coefficient of the reference category and shows the predicted outcome for the reference condition that is not present in the analysis.


In [ ]:
print(result.coef_)
print(result.intercept_)

##### More on Model Evaluation: Confusion Matrix

To further evaluate the model, I generated a confusion matrix using y test and y predicted set. Through the method “classification_report” of sklearn I calculated the following important metrics:
* Precision: ratio of correct positives to total predicted positive (true positives to true positives and false positives)
* Recall: ratio of correct positives to total actual positive (true positives to true positives and false negatives)
* F1-score: a metric that combines precision and recall.

The importance of each metric depends on what your model tries to best accomplish: If it is more important to avoid false positives, precision is the metric to optimize. Alternatively, if it is more important to avoid false negatives, recall is the metric that you want to optimize. And in general, the higher the f1-score, the better your model is. Below, you can see these metrics in each row for each of the subgroups Firmicutes, Actinobacteria and Gammaproteobacteria denoted as 0,1 and 2 in order.

Here is more on sklearn’s confusion matrix : https://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html#sphx-glr-auto-examples-model-selection-plot-confusion-matrix-py

In [ ]:
cnf_matrix = metrics.confusion_matrix(y_test, y_pred)
#calculate precision, recall and f1
metrics.classification_report(y_test, y_pred).split(sep='\n')

Below, I generated the same confusion matrix as a heatmap. It suggests a classification accuracy of 88%. This means that 88% of the times an assembly of a given size and GC content is truly predicted to belong to an outcome subgroup. Respectively, the ERR of 0.12 represents the ratio of misclassified positive and negative classes to all.

In [ ]:
#create heatmap
class_names=['Firmicutes','Actinobacteria','Gammaproteobacteria'] 
plt.figure(1, figsize=(9, 6))
tick_marks = np.arange(len(class_names))
sns.set(font_scale=1.6)
sns.heatmap(pd.DataFrame(cnf_matrix, 
                         index = ['Firmicutes','Actinobacteria','Gammaproteobacteria'] ,
                         columns=['Firmicutes','Actinobacteria','Gammaproteobacteria']), 
                         cmap='Blues', annot=True,fmt='g', annot_kws={"size": 20})
plt.title('Confusion matrix',fontsize=20)
plt.ylabel('Actual', fontsize=14)
plt.xlabel('Predicted',fontsize=14)

In [ ]:
# calculate accuracy (66+49+92)/len(y_test)
metrics.accuracy_score(y_test, y_pred)  

##### More on Model Evaluation: Decision Boundary
Another way to evaluate the performance of my model is to estimate decision boundaries that separate the classes. Given my classification problem with three distinct bacterial subgroups, my function trains three classifiers where each classifier draws a decision boundary for one class vs all other classes. For each class it selects a threshold value above which it assigns values into the class with the highest predicted probability. This plot shows decision boundaries for all three classes with different colors.

In [ ]:
# Plot the decision boundary. Assign a color to each point in the mesh.
X = np.array(X_test)  
Y = np.array(y_test)
x_min, x_max = X[:, 0].min() - .5, X[:, 0].max() + .5
y_min, y_max = X[:, 1].min() - 1.5, X[:, 1].max() + 1.5
#specify step size in the mesh
h = .02  
xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
# make map prediction at every point on the grid
Z = logreg.predict(np.c_[xx.ravel(), yy.ravel()])
# Put the result into a color plot
Z = Z.reshape(xx.shape)

plt.figure(1, figsize=(12, 8))
plt.pcolormesh(xx, yy, Z, cmap=plt.cm.Dark2)

labels = ['Firmicutes','Actinobacteria','Gammaproteobacteria']
colors = [0,4,7]
for i in range(3):
    subset = Y == i
    plt.scatter(X[subset, 0], X[subset, 1], edgecolors='k', s=150, 
                color =plt.cm.Dark2.colors[colors[i]] , label = labels[i])
    
plt.xlabel('Size(MB)', fontsize=14)
plt.ylabel('GC%', fontsize=14)
plt.xlim(xx.min(), xx.max())
plt.ylim(yy.min(), yy.max())
leg = plt.legend(loc = 'upper left',fontsize = 14, framealpha=0.01)

plt.show();

I can also use the “predict_proba” method to get estimates for all classes at a given assembly size(MB) and GC content. For example, with an assembly size of 7 MB and GC content of 50% falling in the grey region, the bacterial subgroup is most probable Gammaproteobacteria since this subgroup has the highest probability for the given features as shown below. 

Here you can find an sklearn example of logistic regression 3 class classifier: https://scikit-learn.org/stable/auto_examples/linear_model/plot_iris_logistic.html.

In [ ]:
logreg.predict_proba([[7,50]])

##### More on Evaluation: ROC curves
Another way to evaluate the output of our classifier is to look at ROC curves. ROC curve for each class is a plot of the class’s true positive rate (Y axis) against its false positive rate (X axis), a trade between sensitivity and specificity. The black line is represents a model that randomly guesses the class and any classifier with a curve above and farther away from the black line is better. The classifier that is closer to the ideal point of a true positive rate of one and a false positive rate of zero approaching the top left corner of the plot is better. This means a steeper ROC curve and also implies a larger area under the curve (AUC).
In order to plot ROC curve for multi-class classification, we need to binarize the output variable using the one-vs-all classification strategy and draw one ROC curve per class. 

Sklearn has built in functions for ROC curves (https://scikit-learn.org/stable/auto_examples/model_selection/plot_roc.html).


In [ ]:
# Use one vs all classifier
classifier = OneVsRestClassifier(LogisticRegression())
y_score = classifier.fit(X_train, y_train).decision_function(X_test)  

# Compute ROC curves for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(3):
    fpr[i], tpr[i], _ = roc_curve(y_test.values == i, y_score[:,i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Plot all ROC curves
plt.figure(1, figsize=(12, 8))

colors = cycle(['aqua', 'darkorange', 'cornflowerblue'])
for i, color in zip(range(3), colors):
    plt.plot(fpr[i], tpr[i], color=color,
             label=f"ROC curve of class {i} is {round(roc_auc[i],2)}")
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC for multi-class')
plt.legend(loc="lower right")
plt.show()

The multiclass logistic regression model suggests that the GC content and assembly size can tease apart the three bacterial subgroups Gammaproteobacteria, Firmicutes and Actinobacteria with a good accuracy. So if more accurate information is missing on potential bacterial members of these subgroups, we could use these features to classify them into their subgroup with an acceptable accuracy. A nice application of logistic regression in bacterial genome studies!